In [21]:
# Windows handles forward slashes correctly in file paths
import json
from pathlib import Path

# Simulate what happens when you read from JSON
json_data = '{"file_name": "data/InfiAgent-DABench/da-dev-tables/abalone.csv"}'
loaded = json.loads(json_data)
file_path_from_json = loaded["file_name"]

print(f"Path from JSON: {file_path_from_json}")

# Convert to Path object - works fine with forward slashes
path_obj = Path(file_path_from_json)
print(f"Path object: {path_obj}")
print(f"Path exists: {path_obj.exists()}")

# If you need native Windows separators, you can get them
print(f"With native separators: {path_obj.as_posix().replace('/', '\\\\')}")

print("\\nKey points:")
print("✓ Windows accepts forward slashes")
print("✓ Path() works with forward slashes") 
print("✓ No conversion needed when reading from JSON")

Path from JSON: data/InfiAgent-DABench/da-dev-tables/abalone.csv
Path object: data\InfiAgent-DABench\da-dev-tables\abalone.csv
Path exists: True
With native separators: data\\InfiAgent-DABench\\da-dev-tables\\abalone.csv
\nKey points:
✓ Windows accepts forward slashes
✓ Path() works with forward slashes
✓ No conversion needed when reading from JSON


In [1]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

load_dotenv(override=True)

questions = Path(os.getenv("QUESTIONS_FILE"))
answers = Path(os.getenv("ANSWERS_FILE"))
# df_json = pd.read_json(questions, lines=True).to_dict(orient='records')
df_questions = pd.read_json(questions, lines=True)
df_answers = pd.read_json(answers, lines=True)
df_merged = df_answers.merge(df_questions, left_on="id", right_on="id", how='inner')

In [2]:
import io
from dataframe_to_dict import parse_dataframe_info

def df_info_to_json(df):
    buffer = io.StringIO()
    df.info(buf=buffer, show_counts=True)
    df_json = parse_dataframe_info(buffer.getvalue())
    return df_json    

In [3]:
from planner import create_plan

path_prefix = Path("data/InfiAgent-DABench/da-dev-tables/")
for index, row in df_merged.iterrows():
    file_name = path_prefix / row['file_name']
    df = pd.read_csv(file_name)
    df_json = df_info_to_json(df)
    plan = create_plan(row['question'], df_json, file_name)
    df_merged.at[index, 'plan'] = plan.model_dump_json()


In [4]:
df_merged.loc[14, 'plan']

'{"task_list":[{"task_name":"Setup Imports and Dependencies","details":"Import all required libraries such as pandas, numpy, sklearn modules, matplotlib, logging, warnings, datetime, and typing.","dependencies":"None","output":"All necessary imports ready","assumptions":""},{"task_name":"Define Configuration Constants","details":"Define all constants including RANDOM_SEED=42, file paths (set to \'data/InfiAgent-DABench/da-dev-tables/my_test_01.csv\'), thresholds (MISSING_THRESHOLD_ROW=0.5, MISSING_THRESHOLD_COL=0.8), and any analysis-specific parameters.","dependencies":"Task 1","output":"Configuration constants defined","assumptions":"Using the file path specified in the user input."},{"task_name":"Setup Logging and Reproducibility","details":"Configure logging with timestamp format, set random seeds (np.random.seed(42)), suppress warnings if needed.","dependencies":"Tasks 1-2","output":"Logging configured, reproducibility ensured","assumptions":""},{"task_name":"Load and Validate Inp

In [4]:
df_merged.to_csv("data/merged_with_plans.csv", index=False)

In [ ]:
from coder import create_code

path_prefix = Path("data/InfiAgent-DABench/da-dev-tables/")
for index, row in df_merged.iterrows():
    file_name = path_prefix / row['file_name']
    df = pd.read_csv(file_name)
    df_json = df_info_to_json(df)
    code = create_code(row['plan'], row['question'], df_json, file_name)
    df_merged.at[index, 'code'] = code.model_dump_json()

In [7]:
df_merged.to_csv("data/merged_with_code.csv", index=False)